# Likelihood and uncertainty
A notebook to illustrate the relationships between the likelihood function and different uncertainty estimates.

## Authors:
**David W. Hogg** (NYU)

## License:
- Copyright 2025 the author. All code is licensed for re-use under the open-source *MIT License*.

## To-do:
- Make something that works.
- Make all plots consistent across all noteboooks, so they are publication-ready.

## Bugs:
- Something is wrong; LF doesn't go down properly at low variance.

In [ ]:
import numpy as np
import pylab as plt
from matplotlib import rcParams
import scipy.optimize as op

In [ ]:
rcParams['figure.figsize'] = [4.0, 4.0]

In [ ]:
# make fake data

N = 23
rng = np.random.default_rng(17) # most random number
true_mean, true_var = np.sqrt(np.e) * np.pi, np.pi ** 2
data = true_mean + np.sqrt(true_var) * rng.normal(size=N)

In [ ]:
# make likelihood function

def ln_oned_gaussian(x, m, v):
    return 0.0 - 0.5 * (x - m) ** 2 / v - 0.5 * np.log(2. * np.pi * v)

def ln_poisson(xs, m, v):
    return np.sum(ln_oned_gaussian(xs, m, v))

def log_likelihood(pars, xs):
    m, v = pars
    return ln_poisson(xs, m, v)

def negative_log_likelihood(pars, xs):
    return -log_likelihood(pars, xs)

In [ ]:
def plot_data(title, pars=None):
    xlim = (-4., 14.)
    plotx = np.linspace(*xlim, 1000)
    if pars is not None:
        m, v = pars
        ys = np.exp(ln_oned_gaussian(plotx, m, v))
        plt.plot(plotx, ys, "r-", lw=2.0, alpha=0.9)
    truey = np.exp(ln_oned_gaussian(plotx, true_mean, true_var))
    plt.plot(plotx, truey, "b-", lw=0.5, alpha=0.5)
    _ = plt.hist(data, bins=32, color="k", density=True)
    plt.xlim(xlim)
    plt.title(title)
    plt.xlabel("x")
    plt.ylabel("fraction per x (density)")

In [ ]:
plot_data("data")

In [ ]:
# find maximum likelihood

res = op.minimize(negative_log_likelihood, (true_mean, true_var), args=(data, ))
print(res)
ml_mean, ml_var = res.x

In [ ]:
plot_data("maximum likelihood fit", pars=(ml_mean, ml_var))

In [ ]:
# make likelihood function image
# bug: loop

dm = 0.2
mlim = (-5., 15.)
mvec = np.arange(mlim[0] + 0.5 * dm, mlim[1], dm)
dv = 0.5
vlim = (0., 20.)
vvec = np.arange(vlim[0] + 0.5 * dv, vlim[1], dv)
print(mvec.shape, vvec.shape)
ms, vs = np.meshgrid(mvec, vvec)
lls = np.zeros_like(ms) + np.nan
for i in range(lls.shape[0]):
    for j in range(lls.shape[1]):
        lls[i, j] = log_likelihood((ms[i, j], vs[i, j]), data)
print(lls)

In [ ]:
mlls = np.max(lls)
plt.imshow(lls, interpolation="nearest", extent=mlim + vlim,
           vmin=mlls - 10., vmax = mlls)
plt.xlabel("mean")
plt.ylabel("variance")